# Basic ideas 

1. Word2vector
2. Next steps
    1. [x] train2 data should be included
    2. [x] a unified model for both task1 and task2

# Packages

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'

sys.path.append(base_dir)
import os
from gensim.similarities.annoy import AnnoyIndexer

from utils import *

import os
import pickle
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
import polars as pl
from annoy import AnnoyIndex
import polars as pl
import polars as pl
from utils import *
from src.eval import model_eval
from src.config import (
    raw_data_session_id_dir, candidate_dir, model_for_eval, candidate_file_name, submit_file_name
    # , w2v_model_file_name, w2v_index_file_name
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config

In [4]:
debug = False

model_for_eval = False

topn = 100

task = 'task1'

version = 'v11'



if task == 'task1':
    target_locals = ['DE', 'JP', 'UK']
elif task == 'task2':
    target_locals = ['ES', 'FR', 'IT']
else:
    assert 1 == 0


model_version = f'w2v_{task}_{version}'
unified_model = True

# target_locals = ['ES', 'FR', 'IT']



if unified_model:
    train_target_locals = ['ES', 'FR', 'IT', "DE", 'JP', 'UK']
else:
    train_target_locals = target_locals
    



if debug:
    n_rows = 100
else:
    n_rows = None

    

# Common setting
# target locales: locales needed for task1
submit_file = os.path.join('../data/sub_files/', 
                           submit_file_name.format(
                                task=task
                               , model_version=model_version
                               , model_for_eval=model_for_eval
                               , topn=topn
                           )
                          )

train_cg_file = os.path.join(base_dir,
                             candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='train'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
eval_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='eval'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
test_cg_file = os.path.join(base_dir,
                            candidate_dir, 
            candidate_file_name.format(
                task=task
                , data_type='test'
                , model_version=model_version
                , model_for_eval=model_for_eval
                , topn=topn
            )
                            )
test4task3_cg_file= os.path.join(base_dir,
                            candidate_dir, 
                                    candidate_file_name.format(
    task=task
    , data_type='test4task3'
    , model_version=model_version
    , model_for_eval=model_for_eval
    , topn=topn
))
print(train_cg_file)
print(eval_cg_file)
print(test_cg_file)
print(test4task3_cg_file)


# model_version = f'w2v_task2_v2'
model_dir = f'../model_training/{model_version}/'

# specific setting
num_tree = 100
# if model_for_eval:
w2v_model_file = os.path.join(model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(model_dir, f"{str(num_tree)}_{model_for_eval}.index")

../data/candidates/task1_train_w2v_task1_v11_False_top100.parquet
../data/candidates/task1_eval_w2v_task1_v11_False_top100.parquet
../data/candidates/task1_test_w2v_task1_v11_False_top100.parquet
../data/candidates/task1_test4task3_w2v_task1_v11_False_top100.parquet


In [5]:
# submit_file_name

In [6]:
! mkdir {model_dir}

mkdir: cannot create directory ‘../model_training/w2v_task1_v11/’: File exists


In [7]:
print(f"{w2v_model_file}; \n {annoy_index_file}; \n {submit_file}  \n {train_cg_file} \n {eval_cg_file} \n {test_cg_file}")

../model_training/w2v_task1_v11/False.model; 
 ../model_training/w2v_task1_v11/100_False.index; 
 ../data/sub_files/submit_task1_w2v_task1_v11_False_top100.parquet  
 ../data/candidates/task1_train_w2v_task1_v11_False_top100.parquet 
 ../data/candidates/task1_eval_w2v_task1_v11_False_top100.parquet 
 ../data/candidates/task1_test_w2v_task1_v11_False_top100.parquet


# Function 

In [8]:
def w2v_rec(target_df, w2vec, annoy_index, topn=100):
    final_cols = ['session_id', 'next_item_prediction']
    def get_next_items(x):
        prev_items = x['prev_items']
        local_rec = x['next_item_prediction']
        final = [ele for ele in local_rec if ele not in prev_items]
        return final
    target_df = (
        target_df.with_columns(
            pl.col('prev_items').apply(lambda x: 
                                       # x.to_list()
                                       list(map(list, zip(*w2vec.wv.most_similar(positive=x.to_list(),
                                                                     topn=topn+10,
                                                                     indexer=annoy_index))))[0]
                                      ).alias('next_item_prediction'))
        .with_columns(
                    pl.struct(["prev_items", "next_item_prediction"]).apply(
                        lambda x: get_next_items(x)).arr.head(topn).alias('next_item_prediction'))
        .select(
          final_cols
        )
        .with_columns(
            pl.col('next_item_prediction').arr.lengths().alias('rec_num')
        )
    )
    return target_df

In [9]:
test_cg_file

'../data/candidates/task1_test_w2v_task1_v11_False_top100.parquet'

In [10]:
# ! ls ../data/candidates/ | grep task2_v2

# Word2vec

## Train data process

## Train model & annnoy index 

In [12]:
train_data = (
    pl.scan_parquet(os.path.join(base_dir,
                                          raw_data_session_id_dir,
                                          'sessions_train1.parquet'),
                             n_rows=n_rows)
        .filter(pl.col('locale').is_in(train_target_locals))
        .with_columns(pl.col('prev_items').apply(str2list))
        .with_columns(pl.col('prev_items').arr.concat(pl.col('next_item')))
)
if not model_for_eval:
    eval_data = (
        pl.scan_parquet(os.path.join(base_dir, 
                                     raw_data_session_id_dir, 
                                     'sessions_eval.parquet'),
                        n_rows=n_rows)
            .filter(pl.col('locale').is_in(train_target_locals))
        .with_columns(pl.col('prev_items').apply(str2list))
        .with_columns(
                    pl.col('prev_items').arr.concat(pl.col('next_item')) )
    )
else:
    eval_data = (
                pl.scan_parquet(os.path.join(base_dir, 
                                     raw_data_session_id_dir, 
                                     'sessions_eval.parquet'),
                        n_rows=n_rows)
            .filter(pl.col('locale').is_in(train_target_locals))
        .with_columns(pl.col('prev_items').apply(str2list))
    )

train2_data = (
    pl.scan_parquet(os.path.join(base_dir,
                                 raw_data_session_id_dir, 
                                 'sessions_train2.parquet'),
                    n_rows=n_rows)
        .filter(pl.col('locale').is_in(train_target_locals))
        .with_columns(pl.col('prev_items').apply(str2list))

)
test_data = (
    pl.scan_parquet(os.path.join(base_dir,
                                           raw_data_session_id_dir, 
                                           f'sessions_test_{task}.parquet'),
                              n_rows=n_rows)
    .with_columns(pl.col('prev_items').apply(str2list))

)

In [13]:
cols_to_keep = ['prev_items']
sentences = (
    pl.concat([train_data.select(cols_to_keep),
               eval_data.select(cols_to_keep),
               train2_data.select(cols_to_keep),
               test_data.select(cols_to_keep)], how='vertical')
).collect().to_dict()['prev_items'].to_list()

# sentences = map(sentences, list)


len(sentences)

3923221

In [14]:
# sentences

In [15]:
# sentences

## W2V config 

In [16]:
vector_size = 32
epochs = 30
sg = 1 # 1 for skip-gram
# pop_thresh = 0.82415
window = 4
min_count = 4

# 

# len(sentences)sentences

In [17]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [ ]:
w2vec = Word2Vec(sentences=sentences, vector_size=vector_size, epochs = epochs, sg=sg,
                 min_count=min_count, workers=14,
                 window=window,
                  compute_loss=True
              , callbacks=[callback()])

Loss after epoch 0: 13866615.0
Loss after epoch 1: 5067301.0
Loss after epoch 2: 3234184.0
Loss after epoch 3: 2412276.0
Loss after epoch 4: 1852344.0
Loss after epoch 5: 1417392.0
Loss after epoch 6: 1157032.0
Loss after epoch 7: 949796.0
Loss after epoch 8: 846516.0
Loss after epoch 9: 721274.0
Loss after epoch 10: 636144.0
Loss after epoch 11: 601656.0
Loss after epoch 12: 535954.0
Loss after epoch 13: 393016.0
Loss after epoch 14: 251620.0
Loss after epoch 15: 244764.0
Loss after epoch 16: 228648.0
Loss after epoch 17: 227316.0
Loss after epoch 18: 220184.0
Loss after epoch 19: 213504.0
Loss after epoch 20: 213464.0
Loss after epoch 21: 201980.0
Loss after epoch 22: 194196.0
Loss after epoch 23: 197308.0


In [ ]:
w2v_model_file

In [ ]:

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(w2vec, num_tree)

## Save Model 

In [ ]:
w2vec.save(w2v_model_file)

In [ ]:
w2v_model_file

In [ ]:

annoy_index.save(annoy_index_file)

## Load model 

In [11]:
w2v_model_file

'../model_training/w2v_task1_v11/False.model'

In [12]:
annoy_index_file

'../model_training/w2v_task1_v11/100_False.index'

In [13]:
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

# Read new data  

In [14]:
target_locals

['DE', 'JP', 'UK']

In [15]:
train2_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train2.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))



## Model Eval

In [14]:
target_locals

['DE', 'JP', 'UK']

In [15]:
eval_candidate_pl = w2v_rec(target_df=eval_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=topn)

In [16]:
# eval_candidate_pl.head(1).collect()

In [17]:
%%time
eval_candidate_pl = eval_candidate_pl.collect()

CPU times: user 7min 52s, sys: 2.65 s, total: 7min 55s
Wall time: 7min 56s


In [18]:
eval_candidate_pl.select('rec_num').describe()

describe,rec_num
str,f64
"""count""",326443.0
"""null_count""",0.0
"""mean""",82.65719
"""std""",37.846632
"""min""",0.0
"""max""",100.0
"""median""",100.0


In [19]:
# model_eval(target_df=eval_pl.join(eval_candidate_pl.lazy(), how='left', on='session_id'))

In [20]:
%%time
# model_eval(target_df=target_df)
eval_final = (
        eval_pl.join(eval_candidate_pl.lazy(), how='left', on='session_id')
        .lazy()
        .with_columns(
            pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
        )
        .with_columns(
            pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
        )
        .with_columns(
            pl.col('mrr').arr.eval(
                pl.arg_where(pl.element()==pl.element().first())
            )
        ).with_columns(
            pl.col('mrr').arr.eval(
                pl.when(pl.element()==0).then(0).otherwise(1/pl.element())
            )
        ).with_columns(
            pl.col('mrr').arr.sum()
            , pl.col('next_item_prediction').arr.head(20).arr.contains(pl.col('next_item')).mean().alias('recall@20')
            , pl.col('next_item_prediction').arr.head(100).arr.contains(pl.col('next_item')).mean().alias('recall@100')
            , pl.col('next_item_prediction').arr.head(topn).arr.contains(pl.col('next_item')).mean().alias('recall@all')


        )
)
final_res = eval_final.select(
        pl.count().alias('total_sessions')
        , pl.col('mrr').mean()
        , pl.col('recall@20').mean()
        , pl.col('recall@100').mean()
        , pl.col('recall@all').mean()

    ).collect()
final_res

CPU times: user 12.5 s, sys: 1.89 s, total: 14.4 s
Wall time: 5.91 s


total_sessions,mrr,recall@20,recall@100,recall@all
u32,f64,f64,f64,f64
326443,0.093158,0.185025,0.312125,0.312125


## Candidate Saving 

### train & eval  

In [16]:
w2vec

In [17]:

train_candidate_pl  = w2v_rec(target_df=train2_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=topn)

In [ ]:
%%time
train_candidate_pl = train_candidate_pl.collect()

In [ ]:
train_candidate_pl.shape

In [ ]:
eval_candidate_pl.filter(pl.col('rec_num')>0).write_parquet(eval_cg_file)

In [ ]:
train_candidate_pl.filter(pl.col('rec_num')>0).write_parquet(train_cg_file)



In [54]:
# reprocess_data = True

### Test 

In [ ]:
test_cg_file

In [ ]:
! ls ../data/candidates | grep 'test_w2v'

In [ ]:
%%time
test_candidate_pl  = w2v_rec(target_df=test_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=topn)
test_candidate_pl = test_candidate_pl.collect()
test_candidate_pl.filter(pl.col('rec_num')>0).write_parquet(test_cg_file)

In [ ]:
# debug_pl = pl.scan_parquet(test_cg_file)

In [ ]:
# debug_pl.head().collect()

In [ ]:
# debug_pl.filter(pl.col('session_id')==4331311).collect()

In [ ]:
# (
#     debug_pl.explode('next_item_prediction')
#         .filter(
#             pl.col('next_item_prediction').is_null()
#         )
# ).collect()#.shape#[0] == 0

### test2task3 

In [ ]:
test4task3_cg_file

In [ ]:
%%time
test4task3_candidate_pl  = w2v_rec(target_df=test4task3_pl,
                   w2vec=w2vec, annoy_index=annoy_index, topn=topn)
test4task3_candidate_pl = test4task3_candidate_pl.collect()
test4task3_candidate_pl.filter(pl.col('rec_num')>0).write_parquet(test4task3_cg_file)

In [ ]:
! ls ../data/candidates/ | grep w2v_task2_v2

In [24]:
# def i2i(df):
#     pl_df = pl.from_dataframe(df)
#     pl_df = (
#         pl_df
#             .with_columns(pl.col('prev_items').apply(lambda row: get_rec(row, annoy_index=annoy_index, topn=100)).alias('next_item_prediction'))
#     )
#     return pl_df

In [ ]:
# # %%time
# def pd_get_rec(df, w2vec, annoy_index, topn):
#     next_item_prediction_lst = []
#     for a in tqdm(df.iterrows(), total=len(df)):
#         prev_items = a[1]['prev_items']
#         res = [ele.replace('[', '').replace(']', '').replace('\n', '').replace("'", '').replace(' ', '') for ele in prev_items.split(' ')]
#         # print(type(res))
#         similarity_dic = w2vec.wv.most_similar(positive=res, topn=topn, indexer=annoy_index)
#         res = [item for item, simi in similarity_dic]
#         next_item_prediction_lst.append(res)
#     df['next_item_prediction'] = next_item_prediction_lst
#     return df

# Validate predictions ✅ 😄

In [33]:
predictions = test_pl.join(test_candidate_pl.lazy(), how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [34]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )

In [35]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(submit_file, engine='pyarrow')

In [36]:
submit_file

'../data/sub_files/submit_task1_w2v_v3_True_top100.parquet'

## Submit to AIcrowd 🚀

In [37]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-1-next-product-recommendation -f {submit_file}

In [16]:
# ! ls {file_path}

als_model.pkl  itemid2item.json  user_item.npz


In [19]:
# eval_pl.head().collect()